# Micromobility D.C. Vehicle Map Creation

In [9]:
import requests
import geopandas as gpd
from shapely.geometry import Point

# Define the API endpoints
api_dict = {
    'Capital Bikeshare': 'https://gbfs.capitalbikeshare.com/gbfs/en/free_bike_status.json',
    'Helbiz': 'https://api.helbiz.com/admin/reporting/washington/gbfs/gbfs.json',
    'Lime Micromobility (including Jump bikes)': 'https://data.lime.bike/api/partners/v1/gbfs/washington_dc/free_bike_status.json',
    'Lyft': 'https://s3.amazonaws.com/lyft-lastmile-production-iad/lbs/dca/free_bike_status.json',
    'Spin': 'https://web.spin.pm/api/gbfs/v1/washington_dc/free_bike_status'
}

# Initialize an empty list to store vehicle information
vehicle_info_list = []

# Iterate over each API endpoint
for company, url in api_dict.items():
    print(f"Processing data from {company}:")
    try:
        response = requests.get(url)
        data = response.json()
        
        if company == 'Helbiz':
            feeds = data.get('data', {}).get('en', {}).get('feeds', [])
            for feed in feeds:
                feed_url = feed.get('url')
                feed_response = requests.get(feed_url)
                feed_data = feed_response.json()
                bikes_data = feed_data.get('data', {}).get('bikes', [])
                
                for bike in bikes_data:
                    lon = bike.get('lon')
                    lat = bike.get('lat')
                    bike_id = bike.get('bike_id')
                    is_reserved = bike.get('is_reserved')
                    is_disabled = bike.get('is_disabled')
                    
                    vehicle_info_list.append({
                        'company': company,
                        'lon': lon,
                        'lat': lat,
                        'bike_id': bike_id,
                        'is_reserved': is_reserved,
                        'is_disabled': is_disabled
                    })
        
        else:
            if 'bikes' in data['data']:
                vehicles_data = data['data']['bikes']
            elif 'vehicles' in data['data']:
                vehicles_data = data['data']['vehicles']
            else:
                print(f"No 'bikes' or 'vehicles' found in the response from {company}")
                continue
            
            # Extract vehicle information
            for vehicle in vehicles_data:
                # Adjust this according to your actual data structure
                lon = vehicle.get('lon')
                lat = vehicle.get('lat')
                bike_id = vehicle.get('bike_id') or vehicle.get('id')  # Adjust if necessary
                is_reserved = vehicle.get('is_reserved')
                is_disabled = vehicle.get('is_disabled')
                # Append vehicle information to the list
                vehicle_info_list.append({
                    'company': company,
                    'lon': lon,
                    'lat': lat,
                    'bike_id': bike_id,
                    'is_reserved': is_reserved,
                    'is_disabled': is_disabled
                })
        
        print(f"Successfully processed data from {company}")
    except Exception as e:
        print(f"Error processing data from {company}: {e}")

# Create a GeoDataFrame from the vehicle information
geometry = [Point(vehicle['lon'], vehicle['lat']) for vehicle in vehicle_info_list]
gdf = gpd.GeoDataFrame(vehicle_info_list, geometry=geometry, crs="EPSG:4326")

# Save the GeoDataFrame as a shapefile
gdf.to_file("new_sat_vehicles.shp")

# Visualize the shapefile on OpenStreetMap using Folium
import folium

# Create a map centered around the first vehicle
m = folium.Map(location=[gdf.geometry.y[0], gdf.geometry.x[0]], zoom_start=12)

# Add the shapefile as a GeoJSON layer
folium.GeoJson(gdf).add_to(m)

# Display the map
m.save("new_sat_vehicles_map.html")


Processing data from Capital Bikeshare:
Successfully processed data from Capital Bikeshare
Processing data from Helbiz:
Successfully processed data from Helbiz
Processing data from Lime Micromobility (including Jump bikes):
Successfully processed data from Lime Micromobility (including Jump bikes)
Processing data from Lyft:
Successfully processed data from Lyft
Processing data from Spin:
Successfully processed data from Spin


/var/folders/82/8hysmmrd67z622hf9p3q3th80000gn/T/ipykernel_84200/4137817841.py:84: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("new_sat_vehicles.shp")


In [11]:
# View the attribute table of the GeoDataFrame
print(gdf.head())  # Display the first few rows
print(gdf.describe())  # Summary statistics of numerical columns
print(gdf.info())  # Information about the GeoDataFrame


             company        lon        lat                           bike_id  \
0  Capital Bikeshare -77.045881  38.909646  617c2b5c789089640f34524e41bbeef5   
1  Capital Bikeshare -77.142283  38.886031  bce571f06694b3a050b30b53d7586815   
2  Capital Bikeshare -77.039374  38.892929  1ca688e1fbaf878604d981df15f46898   
3  Capital Bikeshare -76.999399  38.877904  868702587bcd16b342d3daa3e02b1425   
4  Capital Bikeshare -76.989774  38.977792  ccfa3dc7e625aafa9fd77544eac78e1d   

  is_reserved is_disabled                    geometry  
0           0           0  POINT (-77.04588 38.90965)  
1           0           0  POINT (-77.14228 38.88603)  
2           0           0  POINT (-77.03937 38.89293)  
3           0           0  POINT (-76.99940 38.87790)  
4           0           0  POINT (-76.98977 38.97779)  
                lon           lat
count  15572.000000  15572.000000
mean     -77.023452     38.906211
std        0.035123      0.028817
min      -77.463644     37.553111
25%      -77.

In [13]:
print(gdf['company'].value_counts())

company
Lime Micromobility (including Jump bikes)    7912
Spin                                         4347
Lyft                                         2368
Capital Bikeshare                             904
Helbiz                                         41
Name: count, dtype: int64


In [31]:
# Filter the DataFrame to include only entries where is_reserved is true
reserved_entries = gdf[gdf['is_reserved'] == 1]

# Print the filtered DataFrame
print(reserved_entries)

                                        company        lon        lat  \
909                                      Helbiz -77.234270  38.875402   
912                                      Helbiz -77.030274  38.926266   
914                                      Helbiz -77.234289  38.875340   
931                                      Helbiz -77.009222  38.895455   
933                                      Helbiz -77.234735  38.875433   
944                                      Helbiz -76.743323  39.188643   
1896  Lime Micromobility (including Jump bikes) -76.998000  38.908500   

                                   bike_id is_reserved is_disabled  \
909                              BROKENIOT           1           1   
912                              BROKENIOT           1           0   
914                              BROKENIOT           1           1   
931                              BROKENIOT           1           1   
933                                 EFKMA3           1           

In [33]:
# Save the DataFrame attribute table as a CSV file
gdf.to_csv("mm_attribute_table.csv", index=False)